<a href="https://colab.research.google.com/github/RakeshYadavGit/Fuel-Efficiency-Predictor/blob/main/Fuel_Efficiency_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Loading Dataset

In [4]:
import pandas as pd

columnNames = ["mpg", "cylinders", "displacement", "horsepower", "weight", "acceleration", "model year", "origin"]

url = "https://gist.githubusercontent.com/wmeints/80c1ba22ceeb7a29a0e5e979f0b0afba/raw/auto-mpg.csv"

rowDataset = pd.read_csv(
    url,
    names=columnNames,
    na_values="?",
    comment="\t",
    sep=",",
    index_col=False,
    skiprows=1
)

dataset = rowDataset.copy()
dataset.tail()


/tmp/ipython-input-206390642.py:7: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  rowDataset = pd.read_csv(


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1
397,31.0,4,119.0,82.0,2720,19.4,82,1


In [5]:
dataset.isna().sum()

,0
mpg,0
cylinders,0
displacement,0
horsepower,6
weight,0
acceleration,0
model year,0
origin,0


# Drop the row where are the missing *values*

In [6]:
dataset = dataset.dropna()


# One Hot Encoding: Handling categorical columns

In [7]:
origin = dataset.pop("origin")
dataset["usa"] = (origin==1) * 1.0
dataset["europe"] = (origin==2) * 1.0
dataset["japan"] = (origin==3) * 1.0
dataset.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,usa,europe,japan
393,27.0,4,140.0,86.0,2790,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720,19.4,82,1.0,0.0,0.0


# Dividing dataset into training and testing part

In [8]:
trainingDataset = dataset.sample(frac=0.8, random_state=0)
testingDataset = dataset.drop(trainingDataset.index)

In [9]:
trainStats = trainingDataset.describe()
trainStats

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,usa,europe,japan
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,23.310510,5.477707,195.318471,104.869427,2990.251592,15.559236,75.898089,0.624204,0.178344,0.197452
std,7.728652,1.699788,104.331589,38.096214,843.898596,2.789230,3.675642,0.485101,0.383413,0.398712
min,10.000000,3.000000,68.000000,46.000000,1649.000000,8.000000,70.000000,0.000000,0.000000,0.000000
25%,17.000000,4.000000,105.500000,76.250000,2256.500000,13.800000,73.000000,0.000000,0.000000,0.000000
50%,22.000000,4.000000,151.000000,94.500000,2822.500000,15.500000,76.000000,1.000000,0.000000,0.000000
75%,28.950000,8.000000,265.750000,128.000000,3608.000000,17.200000,79.000000,1.000000,0.000000,0.000000
max,46.600000,8.000000,455.000000,225.000000,5140.000000,24.800000,82.000000,1.000000,1.000000,1.000000


seprate output field from other fields

In [10]:
trainLable = trainingDataset.pop("mpg")
testLable = testingDataset.pop("mpg")

In [11]:
trainStats = trainingDataset.describe()
trainStats = trainStats.transpose()
trainStats

,count,mean,std,min,25%,50%,75%,max
cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
model year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
usa,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


# Model Building

In [13]:
def normalize(x):
    return (x - trainStats["mean"]) / trainStats["std"]
normedTrainDataset = normalize(trainingDataset)
normedTestDataset = normalize(testingDataset)

In [14]:
def buildModel():
  model = keras.Sequential([
      layers.Dense(64, activation="relu", input_shape=[len(trainingDataset.keys())]),
      layers.Dense(64, activation="relu"),
      layers.Dense(1)
  ])
  optimizer = tf.keras.optimizers.RMSprop(0.001)
  model.compile(
      loss="mse",
      optimizer=optimizer,
      metrics=["mae", "mse"]
  )
  return model


In [15]:
model = buildModel()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,865 (19.00 KB)

 Trainable params: 4,865 (19.00 KB)

 Non-trainable params: 0 (0.00 B)

# Train Model

In [16]:
history = model.fit(normedTrainDataset, trainLable, epochs=1000)

Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 600.0932 - mae: 23.1486 - mse: 600.0932  
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 524.0373 - mae: 21.5221 - mse: 524.0373 
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 461.7249 - mae: 20.0435 - mse: 461.7249 
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 390.7468 - mae: 18.3827 - mse: 390.7468  
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 337.5244 - mae: 16.7860 - mse: 337.5244 
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 242.1490 - mae: 14.0257 - mse: 242.1490 
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 182.1216 - mae: 11.7446 - mse: 182.1216 
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 125.5045 - mae: 9.4559 - mse: 125.5045 
Epoch 9/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 83.3763 - mae: 7.3131 - mse: 83.3763 
Epoch 10/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 53.5612 - mae: 5.6908 - mse: 53.

# Model Testing

In [17]:
loss, mae, mse = model.evaluate(normedTestDataset, testLable, verbose=0)
print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

Testing set Mean Abs Error:  1.91 MPG


In [20]:
testPrediction = model.predict(normedTestDataset).flatten()
testPrediction

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([14.230652,  9.229241, 11.490254, 24.257896, 18.646524, 13.292076,
       14.291202, 12.675358, 18.997301, 30.002293, 24.142666, 20.17725 ,
       16.174965, 24.550674, 12.884021, 15.327924, 14.560472, 13.337916,
       16.812424, 12.809348, 12.4619  , 25.030727, 18.376427, 20.722046,
       29.503674, 23.399254, 16.569614, 24.379906, 17.52133 , 16.034685,
       26.702839, 21.580002, 18.431862, 20.727892, 27.497482, 16.747917,
       19.519648, 25.714355, 27.824081, 17.527178, 31.121656, 28.880327,
       14.881444, 37.901836, 35.42093 , 36.33699 , 18.82768 , 19.450043,
       22.096725, 23.258183, 31.456394, 18.731365, 33.121586, 20.009247,
       19.079685, 21.711094, 34.712685, 36.498245, 20.263277, 25.01252 ,
       42.08655 , 33.247005, 25.960283, 25.662533, 30.789719, 37.26063 ,
       35.435505, 35.131508, 36.292133, 30.911667, 19.724113, 32.75272 ,
       28.387957, 29.872978, 37.716568, 33.96439 , 36.249115, 28.355246],
      dtype=float32)

In [21]:
testLable

,mpg
9,15.0
25,10.0
28,9.0
31,25.0
33,19.0
...,...
369,34.0
375,36.0
382,34.0
384,32.0


# Model Conversion for mobile device

In [22]:
convertor = tf.lite.TFLiteConverter.from_keras_model(model)
tfliteModel = convertor.convert()
open("automobile.tflite", "wb").write(tfliteModel)

Saved artifact at '/tmp/tmpmk2pnw8z'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140450082250704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140450082253776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140450082252048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140450082250320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140450082254352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140450082250512: TensorSpec(shape=(), dtype=tf.resource, name=None)


21344